In [1]:
import tensorflow as tf
import numpy as np

from numpy.random import RandomState

In [2]:
## class making done
## safe input and output
class DRRAveStateRepresentation(tf.keras.Model):
    def __init__(self, n_items=5, item_features=100, user_features=100):
        super(DRRAveStateRepresentation, self).__init__()
        ## initialize random_state to 1
        self.random_state = RandomState(1)
        
        ## hold all the parameters variable
        self.n_items = n_items
        self.item_features = item_features
        self.user_features = user_features
        
        ## add to the model parameter
        ## self.attention_weights shape (n_items x 1)
        ## later this need to be reshaped
        self.attention_weights = tf.Variable(initial_value=(0.1 * self.random_state.rand(self.n_items, 1)),
                                             trainable=True,
                                             dtype='float32')
        
    def call(self, user, items):
        '''
        items  : type(tensor) shape = (n_items x item_features)
        user   : type(tensor) shape = (user_features, )
        output : type(tensor) shape = (3 * item_features)
        '''
        ## right will result in numpy array
        ## because there is an issue with tensor matrix multiplications
        right = tf.reshape(tf.transpose(items) @ self.attention_weights, (item_features,))
        middle = user * right
        output = tf.concat([user, middle, right], 0)
        return tf.convert_to_tensor(output)

In [14]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super(AttentionLayer, self).__init__()
        self.units = units
        self.attention_kernel = self.add_weight(shape=(units,1),
                                                initializer='random_uniform',
                                                trainable=True)        
    
    def call(self, inputs):
        return tf.reshape(tf.transpose(inputs) @ self.attention_kernel, (inputs.shape[1],))

## class making done
## safe input and output
class DRRAveStateRepresentation(tf.keras.Model):
    def __init__(self, n_items=5, item_features=100, user_features=100):
        super(DRRAveStateRepresentation, self).__init__()
        ## initialize random_state to 1
        self.random_state = RandomState(1)
        
        ## hold all the parameters variable
        self.n_items = n_items
        self.item_features = item_features
        self.user_features = user_features
        
        ## add to the model parameter
        ## self.attention_weights shape (n_items x 1)
        ## later this need to be reshaped
        self.attention_layer = AttentionLayer(5)
        self.ave_pool = tf.keras.layers.AveragePooling1D(pool_size=2, )
        
    def call(self, user, items):
        '''
        items  : type(tensor) shape = (n_items x item_features)
        user   : type(tensor) shape = (user_features, )
        output : type(tensor) shape = (3 * item_features)
        '''
        ## right will result in numpy array
        ## because there is an issue with tensor matrix multiplications
        right = self.attention_layer(items)
        middle = user * right
        output = tf.concat([user, middle, right], 0)
        output = tf.reshape(output, [1, output.shape[0]])
        return output